### Import the text representation from local module

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

### Import the necessary dependencies to extract the text from the pdf

In [2]:
import numpy as np
import pdfparser.poppler as pdf
from simple_text_representation.classes import Text, Paragraph, Sentence, Word
from simple_text_representation.models import Database, TextModel, ParagraphModel, SentenceModel
from dateutil.parser import parse

### Initialize the Database connection

In [3]:
database = Database('educationalTexts', 'postgres', '', '0.0.0.0', 5432)

''' If the database it's empty, use the following command to create all the tables '''
database.createTables([TextModel, ParagraphModel, SentenceModel])

### Set global variables that describe the structure of the pdf file

In [4]:
INITIAL_PAGE = 5
PAGES_PER_TEXT = 2
GRADE_LVL = 9
ALLOWED_TAGS = ['APRENDEMOS', 'PRACTICAMOS', 'EXTENSIÓN']

In [5]:
def convertPDFtoText(fileName):
    document = pdf.Document(fileName.encode('UTF-8'))

    print('No of pages', document.no_of_pages)
#     for page in document:
#         print('Page', page.page_no, 'size =', page.size)
#         for f in page:
# #             print(' '*1,'Flow')
#             for b in f:
# #                 print(' '*2,'Block', 'bbox=', b.bbox.as_tuple())
#                 for l in b:
#                     print(' '*3, l.text)
#                     #assert l.char_fonts.comp_ratio < 1.0
# #                     for i in range(len(l.text)):
# #                         print(l.text[i].encode('UTF-8'), '(%0.2f, %0.2f, %0.2f, %0.2f)'% l.char_bboxes[i].as_tuple(),\
# #                             l.char_fonts[i].name, l.char_fonts[i].size, l.char_fonts[i].color,)
#                     print()
#                 print("=============================================================================")
    return document

In [6]:
document = convertPDFtoText("../files/comprension_lectora_3.pdf")

No of pages 234


# Limpieza de Textos

## Estructura actual de un Texto

Despues de realizar la estracción de textos con la herramienta PDFtoText se consigue un listado de elementos, donde cada elemento representa una parte de un texto y cada par de elementos adyacentes representan un texto completo. En su estado original, estos traen mucha información innecesaria que debe ser removida y que detallará a continuación.

* Tema a tratar
* Etiqueta de la sección
* Indicaciones del texto
* Titulo del texto
* Referencias de imagenes
* Preguntas relacionas al texto.
* Referencias del texto
* Pie de pagina
* Descripción de la hoja actual.


## Retos de la Limpieza

* Conservar la estructura original del texto, es decir, que todos los parrafos sean identificables.
* Eliminar todas las referencias de imagenes existentes, ya que se usan de forma indeterminada en distintas partes de un texto.
* Adaptar las diferentes estructuras de los archivos PDF provistos

## Pasos de Limpieza

* Usar la Etiqueta de cada texto para determinar el inicio de este.
    * Remover la información previa a la etiqueta
    * Remover la información antes del inicio del texto y despues de la etiqueta
* Eliminar la información extra por hoja.
    * Eliminar la información extra de la primera pagina (referencias y pie de pagina)
    * Eliminar la información extra de la segunda pagina (referencias, preguntas y pie de pagina)
* Eliminar las referencias a imagenes aún existentes.

## Resultado esperado

Finalmente, se espera obtener un arreglo de elementos donde cada represente a un texto completo y que este listo para ser pre-procesado.

In [7]:
def hasAllowedTag(line):
    return any(TAG in line for TAG in ALLOWED_TAGS)

In [8]:
def transformToPython(page):
    listOfFlows = list()
    for flow in page:
        listOfBlocks = list()
        for paragraph in flow:
            listOfLines = ''
            for line in paragraph:
                listOfLines = joinLine(listOfLines, line.text)
#                 listOfLines.append(line.text)
#                 print(line.text)
#                 print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
            listOfBlocks.append(listOfLines)
#             listOfBlocks.append(''.join(listOfLines))
        listOfFlows.append(listOfBlocks)

    return listOfFlows

In [9]:
def joinLine(listOfLines, line):
    newListOfLines = listOfLines
    separator = ''
    
    if (newListOfLines):
        if (newListOfLines[- 1] == '-'):
            newListOfLines = newListOfLines[:-1]
        else:
            separator = ' '
        newListOfLines = newListOfLines + separator + line
    else:
        newListOfLines = newListOfLines + line

    return newListOfLines

In [10]:
def cleanDocument(file):
    nextPageAdded = -1
    listOfTexts = list()
    text = list()
    
    for index,page in enumerate(file):
        if (index > INITIAL_PAGE):
            if (nextPageAdded == 1):
                text.append(transformToPython(page))
                textTuple = (text[0], text[1])
                listOfTexts.append(textTuple)
                nextPageAdded = -1
            else:
                text = list()
                listOfFlows = list()
                for flow in page:
                    listOfBlocks = list()
                    for paragraph in flow:
                        listOfLines = ''
                        for line in paragraph:
                            listOfLines = joinLine(listOfLines, line.text)
#                             print(line.text)
#                             print('========================================')
#                             listOfLines.append(line.text)
                            if (hasAllowedTag(line.text)):
                                nextPageAdded = 1
#                             print(line.text)
#                         listOfBlocks.append(''.join(listOfLines))
                        listOfBlocks.append(listOfLines)
#                         print("------------------------------------------------------")
                    listOfFlows.append(listOfBlocks)
                text.append(listOfFlows)

    return listOfTexts

### List of unformated Texts

In [11]:
listOfTexts = cleanDocument(document)

### Clean conditions

In [12]:
def hasLinkReference(block):
    return 'http' in block

In [13]:
def isBibliographicCitation(block):
    return 'Tomado de' in block

In [14]:
def isPageNumeration(block):
    try:
        int(block)
        return True
    except ValueError:
        return False

In [15]:
def onlyOneCharacter(block):
    return len(block) == 1

In [16]:
def isOnlyDate(block):
    try: 
        parse(block)
        return True
    except ValueError:
        return False

In [17]:
def isStartOfQuestionary(block):
    return ('Responde las preguntas' in block) or \
            ('A partir de la lectura anterior' in block) or \
            ('responde las preguntas' in block) or \
            ('Utiliza el' in block)

In [18]:
def hasIndications(block):
    return 'Lee el siguiente text' in block

In [19]:
def specialCases(block):
    return ('Libro Comunicacion' in block) or \
            ('Comprensión lectora' in block) or \
            ('grado de secundaria' in block)

In [20]:
def cleanContiditions(block):
    return not ( hasLinkReference(block) or \
                onlyOneCharacter(block) or \
                isOnlyDate(block) or \
                specialCases(block) or \
                isPageNumeration(block) or \
                isBibliographicCitation(block) )

In [21]:
def getSecondPage(newPage):
    secondPage = list()

    for block in newPage:
        for line in block:
            secondPage.append(line)
    
    return secondPage

In [22]:
def getTitleAndFirstPage(newPage):
    title = ''
    firstPage = list()

    for i,block in enumerate(newPage):
        for j,line in enumerate(block):
            if (i == 0 and j ==0):
                title = line
            else:
                firstPage.append(line)

    return title, firstPage

In [23]:
def cleanPage(page, isQuestionaryFound = False, isFirstPage = False):
    newPage = list()
    isTagFound = False
    isIndications = False
    questionary = isQuestionaryFound

    for flow in page:
        listOfBlocks = list()
        for block in flow:
            if (cleanContiditions(block)):
                listOfBlocks.append(block)
            isTagFound = isTagFound or hasAllowedTag(block)
            isIndications = isIndications or hasIndications(block)
            questionary = questionary or isStartOfQuestionary(block)
        print(isTagFound, isIndications, questionary, len(newPage) ,listOfBlocks)
        print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
        if questionary:
            break;
        if isTagFound or isIndications:
            newPage = list()
            listOfBlocks = list()
            isTagFound = False
            isIndications = False
        if (len(listOfBlocks) > 0):
            newPage.append(listOfBlocks)

    if (newPage and (isFirstPage and len(newPage) > 1)):
        title, firstPage = getTitleAndFirstPage(newPage)
        return title, firstPage, questionary
    elif (newPage and not isFirstPage):
        return getSecondPage(newPage), questionary
    elif (not newPage and not isFirstPage):
        return [], False
    else:
        return '', [], False

### After declaring all the methods needed to clean the text, we do a simple text with the first two pages

In [24]:
len(listOfTexts)
listOfTexts[0][0]

[['Ficha 1',
  'Leemos textos narrativos',
  '1 APRENDEMOS',
  'Lee el siguiente texto considerando las orientaciones que te brinda el docente.'],
 ['El caballo perdido1',
  'Paulo Coelho',
  'Hace muchos años, en una pobre aldea china, vivía un labrador con su hijo. Su único bien material, aparte de la tierra y de la pequeña casa de paja, era un caballo que había heredado de su padre.',
  'Un buen día el caballo se escapó, dejando al hombre sin animal para labrar la tierra. Sus vecinos, que lo respetaban mucho por su honestidad y diligencia, acudieron a su casa para decirle lo mucho que lamentaban lo ocurrido. Él les agradeció la visita, pero preguntó:',
  '—¿Cómo podéis saber que lo que ocurrió ha sido una desgracia en mi vida?',
  'Alguien comentó en voz baja con un amigo: “Él no quiere aceptar la realidad, dejemos que piense lo que quiera, con tal de que no se entristezca por lo ocurrido”.',
  'Y los vecinos se marcharon, fingiendo estar de acuerdo con lo que habían escuchado.',
  

In [25]:
testTextIndex = 0
mockQuestionary = False
testFirstPage = listOfTexts[testTextIndex][0]
testSecondPage = listOfTexts[testTextIndex][1]

In [26]:
testFirstPage

[['Ficha 1',
  'Leemos textos narrativos',
  '1 APRENDEMOS',
  'Lee el siguiente texto considerando las orientaciones que te brinda el docente.'],
 ['El caballo perdido1',
  'Paulo Coelho',
  'Hace muchos años, en una pobre aldea china, vivía un labrador con su hijo. Su único bien material, aparte de la tierra y de la pequeña casa de paja, era un caballo que había heredado de su padre.',
  'Un buen día el caballo se escapó, dejando al hombre sin animal para labrar la tierra. Sus vecinos, que lo respetaban mucho por su honestidad y diligencia, acudieron a su casa para decirle lo mucho que lamentaban lo ocurrido. Él les agradeció la visita, pero preguntó:',
  '—¿Cómo podéis saber que lo que ocurrió ha sido una desgracia en mi vida?',
  'Alguien comentó en voz baja con un amigo: “Él no quiere aceptar la realidad, dejemos que piense lo que quiera, con tal de que no se entristezca por lo ocurrido”.',
  'Y los vecinos se marcharon, fingiendo estar de acuerdo con lo que habían escuchado.',
  

In [27]:
mockTitle, mockFirstPage, mockQuestionary = cleanPage(testFirstPage, mockQuestionary, True)
print(mockQuestionary, mockTitle, mockFirstPage)

True True False 0 ['Ficha 1', 'Leemos textos narrativos', '1 APRENDEMOS', 'Lee el siguiente texto considerando las orientaciones que te brinda el docente.']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
False False False 0 ['El caballo perdido1', 'Paulo Coelho', 'Hace muchos años, en una pobre aldea china, vivía un labrador con su hijo. Su único bien material, aparte de la tierra y de la pequeña casa de paja, era un caballo que había heredado de su padre.', 'Un buen día el caballo se escapó, dejando al hombre sin animal para labrar la tierra. Sus vecinos, que lo respetaban mucho por su honestidad y diligencia, acudieron a su casa para decirle lo mucho que lamentaban lo ocurrido. Él les agradeció la visita, pero preguntó:', '—¿Cómo podéis saber que lo que ocurrió ha sido una desgracia en mi vida?', 'Alguien comentó en voz baja con un amigo: “Él no quiere aceptar la realidad, dejemos que piense lo que quiera, con tal de que no se entristezca por lo ocurrido”.', 'Y lo

In [28]:
testSecondPage

[['Desconcertados, y pensando que el hombre se estaba volviendo loco, los vecinos se marcharon, comentando por el camino: “¿Será posible que este hombre no entienda que Dios le ha enviado un regalo?”.',
  'Pasado un mes, el hijo del labrador decidió domesticar a la yegua. Pero el animal saltó de una manera inesperada, y el muchacho tuvo una mala caída, rompiéndose una pierna.',
  'Los vecinos retornaron a la casa del labrador, llevando obsequios para el joven herido. El alcalde de la aldea, solemnemente, presentó sus condolencias al padre, diciendo que todos estaban muy tristes por lo que había sucedido.',
  'El hombre agradeció la visita y el cariño de todos. Pero preguntó:',
  '—¿Cómo podéis vosotros saber si lo ocurrido ha sido una desgracia en mi vida?',
  'Esta frase dejó a todos estupefactos, pues nadie puede tener la menor duda de que el accidente de un hijo es una verdadera tragedia. Al salir de la casa del labrador, comentaban entre sí: “Realmente se ha vuelto loco, su único h

In [29]:
mockSecondPage, mockQuestionary = cleanPage(testSecondPage, mockQuestionary)
print(mockQuestionary, mockSecondPage)

False False False 0 ['Desconcertados, y pensando que el hombre se estaba volviendo loco, los vecinos se marcharon, comentando por el camino: “¿Será posible que este hombre no entienda que Dios le ha enviado un regalo?”.', 'Pasado un mes, el hijo del labrador decidió domesticar a la yegua. Pero el animal saltó de una manera inesperada, y el muchacho tuvo una mala caída, rompiéndose una pierna.', 'Los vecinos retornaron a la casa del labrador, llevando obsequios para el joven herido. El alcalde de la aldea, solemnemente, presentó sus condolencias al padre, diciendo que todos estaban muy tristes por lo que había sucedido.', 'El hombre agradeció la visita y el cariño de todos. Pero preguntó:', '—¿Cómo podéis vosotros saber si lo ocurrido ha sido una desgracia en mi vida?', 'Esta frase dejó a todos estupefactos, pues nadie puede tener la menor duda de que el accidente de un hijo es una verdadera tragedia. Al salir de la casa del labrador, comentaban entre sí: “Realmente se ha vuelto loco, s

In [30]:
def mergePages(firstPage, secondPage):
    unformatedPage = firstPage + secondPage
    skipPage = False
    finalPage = list()
    
    for i, block in enumerate(unformatedPage):
        paragraph = ''
        lastChar = block[-1]
        if (not skipPage):            
            if (lastChar == '.'):
                paragraph = block
            else:
                nexBlock = unformatedPage[i+1] if i < len(unformatedPage) - 1 else ''
                paragraph = block + ' ' + nexBlock
                skipPage = True 
            finalPage.append(paragraph)
        else:
            skipPage = False
    
    return finalPage

In [31]:
def transformDocument(listOfUTexts):
    cleanListOfText = list()

    for textTuple in listOfUTexts:
        questionaryFound = False
        firstPage, secondPage = textTuple
        arrParagrahps = []
        text = Text()
        title, cleanFirstPage, questionaryFound = cleanPage(firstPage, questionaryFound, True)
        
        if (cleanFirstPage):
            cleanSecondPage, questionaryFound = cleanPage(secondPage, questionaryFound)
            arrParagrahps = mergePages(cleanFirstPage, cleanSecondPage)
            text.setTitle(title)
            text.setUnformatedParagraphs(arrParagrahps)
            text.formatParagraphs()
            text.setGrade(GRADE_LVL)
            cleanListOfText.append(text)

    return cleanListOfText

In [32]:
results = transformDocument(listOfTexts)

True True False 0 ['Ficha 1', 'Leemos textos narrativos', '1 APRENDEMOS', 'Lee el siguiente texto considerando las orientaciones que te brinda el docente.']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
False False False 0 ['El caballo perdido1', 'Paulo Coelho', 'Hace muchos años, en una pobre aldea china, vivía un labrador con su hijo. Su único bien material, aparte de la tierra y de la pequeña casa de paja, era un caballo que había heredado de su padre.', 'Un buen día el caballo se escapó, dejando al hombre sin animal para labrar la tierra. Sus vecinos, que lo respetaban mucho por su honestidad y diligencia, acudieron a su casa para decirle lo mucho que lamentaban lo ocurrido. Él les agradeció la visita, pero preguntó:', '—¿Cómo podéis saber que lo que ocurrió ha sido una desgracia en mi vida?', 'Alguien comentó en voz baja con un amigo: “Él no quiere aceptar la realidad, dejemos que piense lo que quiera, con tal de que no se entristezca por lo ocurrido”.', 'Y lo

False False False 1 []
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
False False False 1 []
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
False False False 1 []
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
True True False 0 ['Ficha 8', 'Identificando la tesis', '1 APRENDEMOS', 'Lee el siguiente texto considerando las orientaciones que te brinda el docente.']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
False False False 0 ['Carta del Jefe Seattle31', '(Lago Washington, junio de 1854)', 'El Gran Jefe Blanco de Washington nos envía el mensaje de que quiere comprar nuestras tierras. Pero ¿cómo es posible comprar o vender el cielo o el calor de la tierra? Nosotros no comprendemos esta idea. Si no somos dueños de la frescura del aire ni del reflejo del agua, ¿cómo podréis comprarlos? Cada pedazo de esta tierra es sagrado para mi pueblo. Cada rama brillante de un pino, cada puñado de arena de las playas, la

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
False False False 0 ['Los gallinazos sin plumas61', '(Fragmento) Julio Ramón Ribeyro', 'A las seis de la mañana la ciudad se levanta de puntillas y comienza a dar sus primeros pasos. Una fina niebla disuelve el perfil de los objetos y crea como una atmósfera encantada. Las personas que recorren la ciudad a esta hora parece que están hechas de otra sustancia, que pertenecen a un orden de vida fantasmal. Las beatas se arrastran penosamente hasta desaparecer en los pórticos de las iglesias. Los noctámbulos, macerados por la noche, regresan a sus casas envueltos en sus bufandas y en su melancolía. Los basureros inician por la avenida Pardo su paseo siniestro, armados de escobas y de carretas. A esta hora se ve también obreros caminando hacia el tranvía, policías bostezando contra los árboles, canillitas morados de frío, sirvientas sacando los cubos de basura. A esta hora, por último, como a una especie de misteriosa consign

False False False 2 []
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
False False False 2 []
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
False False False 2 []
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
False False False 2 []
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
False False False 0 ['Publicidad 3 91']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
False False False 1 ['Publicidad 4 92']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
False False False 2 []
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
False False False 2 []
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
False False False 2 []
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
False False False 2 []
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
False False False 2 []
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [33]:
results

In [34]:
results[0].getTitle()

'El dengue, la enfermedad “rompe huesos”5'

In [35]:
results[0].toString()

'El dengue es una enfermedad infecciosa muy extendida que se presenta en todas las regiones de clima tropical del planeta. En los últimos años, la transmisión ha aumentado de manera predominante en zonas urbanas y se ha convertido en un importante problema de salud pública. En la actualidad, más de la mitad de la población mundial está en riesgo de contraer la enfermedad.\n1. ¿Qué es el dengue? El mosquito Aedes Aegypti produce el dengue o enfermedad “rompe huesos“.\nEl dengue es una enfermedad viral transmitida por mosquitos, que se caracteriza por producir un importante dolor en las articulaciones y músculos, inflamación de los ganglios y erupción en la piel. No todos los mosquitos contagian el dengue, solo aquellos que previamente han picado a un individuo enfermo.\nExisten cuatro tipos del virus del dengue. La primera vez que una persona es contagiada por cualquiera de estos cuatro virus, adquiere el dengue. Nunca volverá a padecer dengue por el mismo virus; pero si la persona vuel

In [36]:
for index, resultT in enumerate(results):
    resultT.save(database)
    print(index, resultT.toString())
    print('========================================================================')

0 El dengue es una enfermedad infecciosa muy extendida que se presenta en todas las regiones de clima tropical del planeta. En los últimos años, la transmisión ha aumentado de manera predominante en zonas urbanas y se ha convertido en un importante problema de salud pública. En la actualidad, más de la mitad de la población mundial está en riesgo de contraer la enfermedad.
1. ¿Qué es el dengue? El mosquito Aedes Aegypti produce el dengue o enfermedad “rompe huesos“.
El dengue es una enfermedad viral transmitida por mosquitos, que se caracteriza por producir un importante dolor en las articulaciones y músculos, inflamación de los ganglios y erupción en la piel. No todos los mosquitos contagian el dengue, solo aquellos que previamente han picado a un individuo enfermo.
Existen cuatro tipos del virus del dengue. La primera vez que una persona es contagiada por cualquiera de estos cuatro virus, adquiere el dengue. Nunca volverá a padecer dengue por el mismo virus; pero si la persona vuelve

5 Prepárate Verifica de qué material está construido tu hogar, colegio o lugar de trabajo. En caso de ser adobe o autoconstrucción, debes evacuar inmediatamente durante un sismo.
Identifica lugares de protección sísmica, alejados de ventanas y elementos que puedan caerte encima. Asegura los muebles al piso, a los muros o al techo, según corresponda, para que no se vuelquen durante un sismo.
Identifica dónde están las llaves del agua, de corte general de gas y el interruptor o fusible general de electricidad y aprende cómo cortarlas.
Elabora un plan familiar que establezca los puntos de encuentro y los roles de cada integrante del hogar. Mantén un kit de emergencia.
Protégete durante un terremoto Mantén la calma y ubícate en un lugar de protección sísmica.
Protégete y afírmate debajo de un elemento firme. Si no es posible debajo, ubícate junto a él.
Si estás en silla de ruedas, intenta moverte a un lugar de protección sísmica. Si no es posible, frénala y cubre tu cabeza y cuello con tus

9 (Adaptación) Hoy hablar de un trasplante de corazón no supone mayor asombro, pero en 1967 el mundo entero quedó con la boca abierta cuando en Sudáfrica se realizó la primera cirugía exitosa de trasplante cardiaco. El cirujano detrás de una de las hazañas médicas más importantes de los últimos 50 años fue Christiaan Barnard.
El joven Christiaan decidió hacerse cardiólogo tras sufrir la pérdida de uno de sus cuatro hermanos de solo cinco años, víctima de un problema cardíaco. Fue licenciado en Medicina en el hospital Groote Schuur de Ciudad del Cabo y siendo ya cirujano se especializó en cardiología gracias a una beca de dos años en la Universidad de Minneapolis (EE. UU.), en la clínica del doctor Owen. Sería allí donde se introduciría en los secretos de la ciencia cardiovascular, iniciando con la técnica de trasplantes de corazón en animales.
“Piensa en grande y tus hechos crecerán; piensa en pequeño y quedarás atrás; piensa que puedes y podrás, todo está en el estado mental”.
Tras qu

11 Moby Dick45 (Fragmento) Herman Melville.
Mi nombre es Ismael. Hace unos años, encontrándome sin apenas dinero, se me ocurrió embarcarme y ver mundo. Pero no como pasajero, sino como tripulante, como simple marinero de proa. Esto al principio resulta un poco desagradable, ya que hay que andar saltando de un lado a otro, y lo marean a uno con órdenes y tareas desagradables, pero con el tiempo se acostumbra uno. Y por supuesto, porque se empeñan en pagarme mi pasaje, mientras que un pasajero se ha de pagar el suyo. Aún hay más: me gusta el aire puro y el ejercicio saludable. Digamos que el marinero de proa recibe más cantidad de aire puro que los oficiales, que van a popa y reciben el aire ya de segunda mano.* Por último diré que había decidido embarcarme en un ballenero, ya que las ballenas me atraían irresistiblemente. Cierto que resulta una caza peligrosa, pero tiene sus compensaciones: los mares en los que esos cetáceos se mueven, la maravillosa espera, el grito coral cuando se enc

13 (Adaptación) Ante la partida eterna de nuestro compañero en El Comercio, rescatamos una crónica de 1995, ejemplo de su calidad de periodista.
Sentado en la vereda de una gasolinera de Casma, el periodista Javier Ascue Sarmiento, que aún escribía a máquina con dos dedos, piensa que lo van a despedir. Le habían ordenado hacer guardia en el Grupo 8 de la Fuerza Aérea, desde donde se iban a transportar provisiones hacia Chimbote y el Callejón de Huaylas, las zonas más perjudicadas con el terremoto del día anterior. Pero Ascue había convencido al fotógrafo José Michilot para subirse al avión en vez de regresar a El Comercio.
Cuando llegaron a Chimbote, se dio cuenta de que cuatro periodistas de esta casa y de todos los diarios ya estaban allí; y que Ascue, novato redactor de 25 años, no tenía nada que hacer, cosa realmente imposible para un buen periodista. “Todos los periodistas se han ido a Huaraz por un camino demasiado largo”, lo tienta el policía mostrándole un mapa. Incorregible, A

15 Los alimentos transgénicos son alimentos que son tratados genéticamente para su consumo. A diferencia de los que se cultivan y se producen de manera natural, a estos alimentos se les modifica el ADN creando mutaciones y generando un producto con unas características determinadas, eliminando su proceso natural.
Suelen ser alimentos de origen vegetal, aunque en la actualidad se han comenzado algunos procesos también de tipo animal. Como ejemplos concretos de este tipo de cultivos, se pueden encontrar algunas frutas y verduras en las que se eliminan las pepitas, se modifica el color y el sabor, se retarda el proceso de maduración, se incluyen mayor cantidad de minerales o, incluso, se crean híbridos entre varios tipos de alimentos vegetales, creando nuevos productos.
La soya y el maíz son dos de los alimentos de origen vegetal que más modificaciones genéticas han sufrido, mejorando su composición para que sean más resistentes a las inclemencias del tiempo y a la exposición a diferentes

18 Alguna vez me pasó, y eso fue suficiente para entender mi mejor lección: ¡No te enamores de tu mejor amiga!, hacerlo es un gravísimo error que lamentarás tarde o temprano.
¿Quién dice que el amor entre mejores amigos es un error? Discrepo profundamente de esa idea, al punto de afirmar que lo mejor que puede pasarte es que te enamores de tu mejor amigo.
Recuerdo las veces que compartimos tantas cosas que hoy forman parte del pasado. Éramos tan solo dos niños que vivían en la misma cuadra. Ella siempre tan risueña; yo, un muchacho travieso y callejero, el “Mataperros” como me decían en la cuadra.
Nosotros nos conocimos desde muy niños, y creo que quien se enamoró primero fui yo. ¿Y él?, para nada, ni se lo imaginaba; solo me demostraba un cariño de amigos. Era tanta la confianza entre los dos que tuve que escuchar sus historias y romances con otras chicas, lo que se contaban en las reuniones de “patas”, e incluso llegué a sentir cólera por mis propios amigos, esos que lo invitaban a s

20 Coplas a la muerte de su padre72 Recuerde el alma dormida, avive el seso y despierte contemplando cómo se pasa la vida, cómo se viene la muerte tan callando, cuán presto se va el placer, cómo, después de acordado, da dolor; cómo, a nuestro parecer, cualquiera tiempo pasado fue mejor.
Jorge Manrique A partir de los versos leídos se puede inferir:.
a. Cómo recordar el placer da dolor.
b. Valorar la vida porque pasa muy rápido.
c. Todo tiempo pasado fue mejor.
d. Debemos vivir el presente y no el pasado.
Según el contexto del verso, la palabra presto se podría reemplazar por.
cuán presto se va el placer, cómo, después de acordado, da dolor; a. Rápido.
b. Despacio c. Lento.
d. Calmado El tema que presenta el poema es:.
a. Recuerdo del alma dormida.
b. La fugacidad de la vida.
c. La pérdida de un ser querido.
d. Aprovechar y vivir el momento.
¿Cuál es el tono del autor empleado en el poema? a. Quejumbroso.
b. Serio c. Agresivo.
d. Melancólico ¿Cuál es la intención del autor al utilizar l

27 No derrumben mi casa vieja, había dicho. No derrumben mi casa.
II Teníamos nuestra pérgola, y dos puertas a la calle, un jardín a la entrada, pequeño pero grande, un manzano que yace seco ahora por el grito y el cemento. El durazno y el naranjo habían muerto anteriormente, pero teníamos también (¡cómo olvidarlo!) un árbol de granadas. Granadas que salían de su tronco, rojas,.
IV verdes, el árbol se mezclaba con el muro, y al lado, en la calle, un tronco que daba moras cada año, que llenaba de hojas en otoño las puertas de mi casa.
III No derrumben mi vieja casa, había dicho, dejen al menos mis granadas y mis moras, mis manzanas y mis rejas.
Todo esto contenía mi pequeño jardín. Era un pedazo de tierra custodiado día y tarde por una verja, una reja castaña y alta que los niños a la salida del colegio saltaban fácilmente, llevándose las manzanas y las moras, las granadas y las flores.
Es cierto, no lo niego, las paredes se caían y las puertas no cerraban totalmente. Pero mataron mi ca